## Домашнее задание к лекции «Продвинутый pandas»

Преподаватель: Константин Башевой

### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

* если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
* для источников paid и email из России ставим ad;
* для источников paid и email не из России ставим other;
* все остальные варианты берём из traffic_source без изменений.

In [2]:
import pandas as pd

Загружаем данные и изучаем

In [6]:
table = pd.read_csv('visit_log.csv', sep=';')
table.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [7]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       18938 non-null  int64 
 1   visit_id        18938 non-null  object
 2   url             18938 non-null  object
 3   region          18938 non-null  object
 4   user_id         18938 non-null  object
 5   traffic_source  18938 non-null  object
dtypes: int64(1), object(5)
memory usage: 887.8+ KB


Создаем функцию для определения значения source_type

In [10]:
def get_source_type(row):
    """
    Возвращает значение столбца 'source_type' на основе значения столбца 'traffic_source' 
    и 'region' в переданной строке данных.

    Параметры:
    - row: строка данных (запись), содержащая значения столбцов 'traffic_source' и 'region'

    Возвращаемое значение:
    Значение столбца 'source_type' на основе условий:

    - Если значение столбца 'traffic_source' равно 'yandex' или 'google', возвращает 'organic'.
    - Если значение столбца 'traffic_source' равно 'paid' или 'email' и значение столбца 'region' равно 'Russia', возвращает 'ad'.
    - Если значение столбца 'traffic_source' равно 'paid' или 'email' и значение столбца 'region' не равно 'Russia', возвращает 'other'.
    - В противном случае, возвращает значение столбца 'traffic_source'.
    """
    if row['traffic_source'] in ['yandex', 'google']:
        return 'organic'
    elif row['traffic_source'] in ['paid', 'email']:
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return row['traffic_source']

Применяем функцию `get_source_type`

In [15]:
table['source_type'] = table.apply(get_source_type, axis=1)
table.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


Удаляем дубликаты, чтобы наглядно посмотреть как подставились значения.

In [22]:
unique_values = table[['traffic_source', 'source_type']].drop_duplicates()
print(unique_values)

   traffic_source source_type
0          yandex     organic
1          direct      direct
6            paid          ad
9           email          ad
11         google     organic
15           paid       other
20          email       other


Все сработало!

### Задание 2
В файле `URLs.txt` содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод `str.contains` и регулярное выражение в соответствие с заданным шаблоном.

In [58]:
import pandas as pd
import re

Загружаем файл `URLs.txt` в датафрейм и читаем содержимое

In [46]:
df = pd.read_csv('URLs.txt') 
df.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [56]:
news_df = df[df['url'].str.contains(r"\/\d{8}-", regex=True)]
news_df.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


Эта строка фильтрует исходный датафрейм `df` на основе условия, заданного внутри квадратных скобок (`[ ]`), и создает новый датафрейм `news_df` с отфильтрованными данными.

Внутри скобок (`[ ]`) находится условие фильтрации:

df['url'].str.contains(r"\/\d{8}-", regex=True)


- `df['url']` обращается к столбцу 'url' в исходном датафрейме `df`.
- `.str.contains()` является методом строки, который проверяет, содержит ли каждое значение в столбце 'url' заданное регулярное выражение или подстроку.
- `r"\/\d{8}-"` - это регулярное выражение, используемое для поиска подстроки в тексте, соответствующей определенному шаблону. В данном регулярном выражении используются следующие элементы:

- `r` перед кавычками, указывает на использование "сырой" строки ("raw string"), что означает, что обратная косая черта `\` не будет интерпретироваться как специальный символ.
- `\/` - это обратная косая черта `\`, за которой следует символ `/`. Обратная косая черта используется для экранирования символа `/`, чтобы он был интерпретирован как обычный символ, а не как разделитель шаблона.
- `\d` - это специальный символ, который соответствует любой цифре от 0 до 9.
- `{8}` - это фигурные скобки, указывающие на количество повторений предыдущего элемента. В данном случае, `\d{8}` означает, что мы ожидаем точно 8 цифр.
- `-` - это символ тире, который ожидается после 8 цифр.

Совместно все вышеперечисленные элементы образуют шаблон, который ищет подстроку, состоящую из символа `/`, за которым идут точно 8 цифр, а затем символ `-`. 

Примеры строк, которые могут соответствовать этому шаблону:

- `/politics/36188461-`
- `/world/36007585-`
- `/science/36157853-`
- `/video/36001498-`

- `regex=True` указывает на использование регулярных выражений при поиске.

Поэтому эта строка означает, что новый датафрейм `news_df` будет содержать только те строки из исходного датафрейма `df`, где столбец 'url' содержит подстроку, соответствующую заданному регулярному выражению.

### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [60]:
import pandas as pd

In [61]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [63]:
user_ratings_count = ratings.groupby('userId')['rating'].count().reset_index()
user_ratings_count.head()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [65]:
active_users = user_ratings_count[user_ratings_count['rating'] > 100]
active_users.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [66]:
user_timestamps = ratings.groupby('userId')['timestamp'].agg(['min', 'max']).reset_index()
user_timestamps.head()

,userId,min,max
0,1,1260759108,1260759205
1,2,835355395,835356246
2,3,1298861589,1298932787
3,4,949778714,949982274
4,5,1163373044,1163375145


In [67]:
active_users_timestamps = pd.merge(active_users, user_timestamps, on='userId')
active_users_timestamps.head()

,userId,rating,min,max
0,4,204,949778714,949982274
1,8,116,1154389340,1154474527
2,15,1700,997937239,1469330735
3,17,363,1127468587,1127476640
4,19,423,855190091,855195373


In [69]:
active_users_timestamps = pd.merge(active_users, user_timestamps, on='userId')
active_users_timestamps.head()

,userId,rating,min,max
0,4,204,949778714,949982274
1,8,116,1154389340,1154474527
2,15,1700,997937239,1469330735
3,17,363,1127468587,1127476640
4,19,423,855190091,855195373


In [70]:
active_users_timestamps['lifetime'] = active_users_timestamps['max'] - active_users_timestamps['min']
active_users_timestamps.head()

,userId,rating,min,max,lifetime
0,4,204,949778714,949982274,203560
1,8,116,1154389340,1154474527,85187
2,15,1700,997937239,1469330735,471393496
3,17,363,1127468587,1127476640,8053
4,19,423,855190091,855195373,5282


In [72]:
average_lifetime = active_users_timestamps['lifetime'].mean()
print(average_lifetime)


40080507.4496124
